In [1]:
'''

'''
import os
import numpy as np
import json 
import math 
import random
import time
import GAMN as ga

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Add, ELU
from tensorflow.keras.regularizers import l1_l2, l1,l2
from tensorflow.keras.optimizers import Adam

import plotly as py
import plotly.offline as offline
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.io as pio

offline.init_notebook_mode(connected=True) 
CURRENT_MODEL_SAVE_DIR = './save/'
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES']= '-1' #-1 for CPU, 0 for GPU
tf.__version__ , tf.keras.__version__, tf.config.list_physical_devices('GPU')

('2.2.0-rc3', '2.3.0-tf', [])

In [2]:
def add_trace(subplot,trace,row,col):
    for d in trace.data:
        subplot.add_traces(d, rows=[row], cols=[col])
    return

def autoModel(enCode,deCode):
    inputDim = enCode.layers[0].input_shape[0][1]
    input_layer = Input((inputDim,))
    X = enCode(input_layer)
    output_layer = deCode(X)   
    return Model(input_layer, output_layer)

In [3]:
'''
Load x, y training data

'''
nextStep = 1
fullPath = CURRENT_MODEL_SAVE_DIR + 'CChar0.json'   
with open(fullPath, 'r') as fp:
    mList = json.load(fp)  
    
char_stroke_data = np.array(mList)
char_target = ga.flatten(char_stroke_data)

char_stroke_data.shape, char_target.shape

((4, 52, 2), (52, 8))

In [4]:
OutStroke = char_stroke_data
gs0 = np.array([OutStroke[0,:,0], OutStroke[0,:,1]]) #x,y Stroke 0
gs1 = np.array([OutStroke[1,:,0], OutStroke[1,:,1]]) #x,y
gs2 = np.array([OutStroke[2,:,0], OutStroke[2,:,1]]) #x,y
gs3 = np.array([OutStroke[3,:,0], OutStroke[3,:,1]]) #x,y

layout = go.Layout(
    height = 800,
    width = 800,
    hovermode= 'closest',
    showlegend=False,
)

x_input_graph = ga.graphCurves([gs0,gs1,gs2,gs3,], layout, dashRange=range(40,80), mode='markers')
x_input_graph

In [5]:
# x_input_graph.write_image("image/figCharTarget.png")

In [6]:
z_input,z_target,x_target = ga.get_circle_input_target(char_data=char_target)
uv = z_target-z_input

x= z_input[:,0]
y= z_input[:,1]
u = uv[:,0]
v = uv[:,1]

qt = ff.create_quiver(x, y, u, v,
        scale=.5,
        arrow_scale=.5,
        name='quiver',
      )
qt.layout = layout
qt.layout.colorway=['blue']+py.colors.qualitative.Dark24
qt

In [7]:
'''
Build Models

'''
##########################################
training = False
cnt = 0
np.random.seed(1)
tf.random.set_seed(1)

nPoints=300
R=.4
r=.25
r_offset=.06
t_offset=0.017
num_twists=5
phi_offset = 0

dDims = [2,8,14,20]
eDims = [20,14,8,2]
gDims = [20,14,10,8]

eModel = ga.mlpModel(eDims)
dModel = ga.mlpModel(dDims)
gModel = ga.mlpModel(gDims)

zaModel = ga.pairModel(dModel, eModel)
ganModel = ga.pairModel(dModel, gModel)

# eModel.summary()
# dModel.summary()
# zaModel.summary()
# ganModel.summary()

In [8]:
'''
Compile model with learning rate

'''
za_learning_rate=0.0001
za_learning_rate=0.00005
# za_learning_rate=0.00001

ga.compile_model(zaModel,za_learning_rate*.5)
ga.compile_model(ganModel,za_learning_rate)
print('Number of Points:{} Twists:{} R:{}  r:{}  r offset:{}  t offset:{} phi offset:{}'.format(
    nPoints, num_twists, R, r, r_offset, t_offset, phi_offset))

Number of Points:300 Twists:5 R:0.4  r:0.25  r offset:0.06  t offset:0.017 phi offset:0


In [9]:
'''
train/fit model
get_multi_helix_data(nPoints=300, r=.04, num_twists=10

'''
z_lossLst = []
z_maeLst = []
x_lossLst = []
x_maeLst = []

z_mae=0
z_min_mae = 100
z_max_mae = -1

x_mae=0
x_min_mae = 100
x_max_mae = -1

epochs, batch_size, repeats = 1,1,3000
epochs, batch_size, repeats = 1,1,1
if training:
    start = time.time()
    z_input,z_target,x_target = ga.get_circle_input_target(char_data=char_target)
    for rr in range(repeats):
        zaModel.fit(z_input, z_target, epochs=epochs, batch_size=batch_size ,verbose=0)
        ganModel.fit(z_input, x_target, epochs=epochs, batch_size=batch_size ,verbose=0)

        if (rr) % (repeats/10) == 0:
            [z_loss, z_mae] = zaModel.evaluate(z_input, z_target,verbose=0)
            [x_loss, x_mae] = ganModel.evaluate(z_input, x_target,verbose=0)
            print("cnt {0:,.0f} mae z:{1:.5f} x:{2:.5f}".format(cnt+epochs*rr, z_mae, x_mae))           
            z_lossLst.append(z_loss)
            z_maeLst.append(z_mae)
            x_lossLst.append(x_loss)
            x_maeLst.append(x_mae)

    z_meanLoss = (sum(z_lossLst) / len(z_lossLst))*2000   
    z_mae = (sum(z_maeLst) / len(z_maeLst))*1000
    z_min_mae= np.min(z_maeLst)
    z_max_mae= np.max(z_maeLst)

    x_meanLoss = (sum(x_lossLst) / len(x_lossLst))*1000   
    x_mae = (sum(x_maeLst) / len(x_maeLst))*1000
    x_min_mae= np.min(x_maeLst)
    x_max_mae= np.max(x_maeLst)

    cnt += epochs*repeats    
    print("lr: {0:.7f} cnt: {1:,.0f} mae z:({2:.5f} : {3:.5f}) x:({4:.5f} : {5:.5f})".format(
        za_learning_rate, cnt, z_min_mae, z_max_mae,x_min_mae, x_max_mae,))

    print('time: {0:.2f}'.format( time.time()-start))    

else:
    eModel.load_weights('./save/encode 2D circle and char target')
    dModel.load_weights('./save/decode 2D circle and char target')
    gModel.load_weights('./save/gen 2D circle and char target')
    zaModel = autoModel(dModel,eModel)
    ganModel = autoModel(dModel,gModel)
    
#     zaModel.summary(), ganModel.summary()

In [10]:
'''
dDims = [2,8,14,20]
eDims = [20,14,8,2]
gDims = [20,14,10,8]
lr: 0.0001000 cnt: 2,000 mae z:(0.00466 : 0.87862) x:(0.00831 : 0.42940)
lr: 0.0001000 cnt: 5,000 mae z:(0.00263 : 0.00340) x:(0.00466 : 0.00790)

dDims = [2,9,18,20]
eDims = [20,18,9,2]
gDims = [20,18,10,8]
lr: 0.0001000 cnt: 3,000 mae z:(0.00322 : 1.15893) x:(0.00674 : 0.49994)
lr: 0.0000500 cnt: 5,000 mae z:(0.00164 : 0.00181) x:(0.00527 : 0.00621)


eDims = [24,20,8,3,2]
dDims = [2,3,8,20,24]
gDims = [24,20,12,8]
lr: 0.0001000 cnt: 1,000 mae z:(0.02158 : 0.58188) x:(0.01110 : 0.42270)
lr: 0.0001000 cnt: 3,000 mae z:(0.00322 : 1.15893) x:(0.00674 : 0.49994)

dDims = [2,4,6]
eDims = [6,4,2]
gDims = [6,12,10,8]
lr: 0.0001000 cnt: 1,000 mae z:(0.03452 : 0.65346) x:(0.01679 : 0.44770)

'''
z_input.shape,z_target.shape,x_target.shape

((104, 2), (104, 2), (104, 8))

In [11]:

######################
d_hat = dModel.predict(z_input)
z_hat = eModel.predict(d_hat)

x_hat = gModel.predict(d_hat)
x_stroke = ga.unFlatten(x_hat)

########### z input
zd_in = np.array([z_input[:,0], z_input[:,1]]) #x,y
zInGraph = ga.graphCurves([zd_in], layout, dashRange=range(40,80), mode='lines')
zInGraph.data[0].line = dict(width=2)

########### z output
zOutG = np.array([z_hat[:,0], z_hat[:,1]]) #x,y
zOut_graph = ga.graphCurves([zOutG], layout, dashRange=range(40,80), mode='lines')
zOut_graph.data[0].line = dict(width=2)

############ char output
OutStroke = d_hat # x_stroke
gs0 = np.array([x_hat[:,0], x_hat[:,1]]) 
gs1 = np.array([x_hat[:,1], x_hat[:,2]]) 
gs2 = np.array([x_hat[:,2], x_hat[:,3]]) 
gs3 = np.array([x_hat[:,3], x_hat[:,4]]) 
gs4 = np.array([x_hat[:,4], x_hat[:,5]]) 
x_graph = ga.graphCurves([gs0,gs1,gs2,gs3,gs4], layout, dashRange=range(40,80), mode='lines')

############# 
subplot_titles=["input", "encode(z_input)", "decode(encode(z_input))"]
pp = py.subplots.make_subplots(rows=1, cols=3, subplot_titles=subplot_titles) #,shared_xaxes=True,shared_yaxes=True,)
pp.add_traces(zInGraph.data[0], rows=[1], cols=[1])

pp.add_traces(x_graph.data[0], rows=[1], cols=[2])
pp.add_traces(x_graph.data[1], rows=[1], cols=[2])
pp.add_traces(x_graph.data[2], rows=[1], cols=[2])
pp.add_traces(x_graph.data[3], rows=[1], cols=[2])
pp.add_traces(x_graph.data[4], rows=[1], cols=[2])

pp.add_traces(zOut_graph.data[0], rows=[1], cols=[3])
pp.update_layout(template='plotly_white',height=400, width=1000) #height=800, width=600, 
pp

In [12]:
# x_stroke = ga.unFlatten(x_hat)
# OutStroke = x_stroke

# # OutStroke = ufd #char_stroke_data
# gs0 = np.array([OutStroke[0,:,0], OutStroke[0,:,1]]) #x,y Stroke 0
# gs1 = np.array([OutStroke[1,:,0], OutStroke[1,:,1]]) #x,y
# gs2 = np.array([OutStroke[2,:,0], OutStroke[2,:,1]]) #x,y
# gs3 = np.array([OutStroke[3,:,0], OutStroke[3,:,1]]) #x,y

# x_output_graph = ga.graphCurves([gs0,gs1,gs2,gs3,], layout, dashRange=range(40,80), mode='lines')
# x_output_graph

In [13]:
# x_output_graph.write_image("image/fig4c.png")

In [14]:
'''
data_input, data_target

'''
########################
x= z_input[:,0]
y= z_input[:,1]
uv = z_target-z_input
u=uv[:,0]
v=uv[:,1]

fqTarget = ff.create_quiver(x, y, u, v,
        scale=.6,
        arrow_scale=.6,
        name='in to target',
        line=dict(width=1)                                
      ).data[0]

go.Figure(data=[fqTarget],layout=layout)

In [15]:
d = dModel(z_input)
xx_hat =gModel(d)
xx_stroke = ga.unFlatten(xx_hat)

xy = xx_stroke[0,:,0:2]
uv = xy[1:]-xy[:-1]
x= xy[:-1,0]
y= xy[:-1,1]
u = uv[:,0]
v = uv[:,1]
fq0 = ff.create_quiver(x, y, u, v,
        scale=.6,
        arrow_scale=.6,
        name='quiver',
      )
xy = x_stroke[1,:,0:2]
uv = xy[1:]-xy[:-1]
x= xy[:-1,0]
y= xy[:-1,1]
u = uv[:,0]
v = uv[:,1]
fq1 = ff.create_quiver(x, y, u, v,
        scale=.6,
        arrow_scale=.6,
        name='quiver',
      )
xy = x_stroke[2,:,0:2]
uv = xy[1:]-xy[:-1]
x= xy[:-1,0]
y= xy[:-1,1]
u = uv[:,0]
v = uv[:,1]
fq2 = ff.create_quiver(x, y, u, v,
        scale=.6,
        arrow_scale=.6,
        name='quiver',
      )
xy = x_stroke[3,:,0:2]
uv = xy[1:]-xy[:-1]
x= xy[:-1,0]
y= xy[:-1,1]
u = uv[:,0]
v = uv[:,1]
fq3 = ff.create_quiver(x, y, u, v,
        scale=.6,
        arrow_scale=.6,
        name='quiver',
      )

fq0.layout = layout
fq0.layout.colorway=['lightskyblue']+py.colors.qualitative.Dark24
fq0.add_traces(fq1.data[0])
fq0.add_traces(fq2.data[0])
fq0.add_traces(fq3.data[0])
fq0


In [16]:
xx,yy = np.mgrid[-.5:1.5:20j, -.5:1.5:20j]
s = xx.shape[0]*yy.shape[0]
xyz = np.column_stack((
    xx.ravel(),
    yy.ravel(),
#     np.ones([s])*.5
))

dOut = dModel(xyz)
eOut = eModel(dOut)
x= xyz[:,0]
y= xyz[:,1]
uv = eOut-xyz
u = uv[:,0]
v = uv[:,1]

fq = ff.create_quiver(x, y, u, v,
        scale=.4,
        arrow_scale=.4,
        name='quiver',
      )

fq.layout = layout
fq.layout.colorway=['lightskyblue']+py.colors.qualitative.Dark24
fq.add_traces(fqTarget)
fq

In [17]:
# fq0.write_image("image/figp1.png")

In [18]:
# notebookName='2D circle and char target'

In [19]:
# fullPath = CURRENT_MODEL_SAVE_DIR + 'encode ' + notebookName  
# eModel.save_weights(fullPath)

# fullPath = CURRENT_MODEL_SAVE_DIR + 'decode ' + notebookName  
# dModel.save_weights(fullPath)

# fullPath = CURRENT_MODEL_SAVE_DIR + 'gen ' + notebookName  
# gModel.save_weights(fullPath)

# fullPath
# './save/gen 2D circle and char target'

In [20]:
# blendout = np.array([z_hat[:300],z_hat[:300]])
# with open(CURRENT_MODEL_SAVE_DIR +'blendout.json', 'w') as fp:
#     json.dump(blendout.tolist(),fp)
# gOut.shape

In [21]:
'''
Create recursive paths through attractor

'''
num = 20
xpos = .55
ypos = .46
x = .04*np.cos(np.linspace(0,math.pi*2,num))+xpos
y = .04*np.sin(np.linspace(0,math.pi*2,num))+ypos

z_start_list = np.stack([x,y],1)
z_path_list = ga.follow_z_path_2d(eModel,dModel,z_start_list,num_obs=40) 
gl = ga.graph_path_list_2d(z_path_list)
f = go.Figure(data=gl,layout=layout)
f.layout.title='xpos: {} ypos:{}'.format(xpos, ypos)
f
# f.write_image("./image/2D attractor vortex.png")

In [22]:
'''
Graph output character based on recurrent path path_index
'''
layout2D = go.Layout(
    width = 600,
    height = 800,
    title = '',
    showlegend=True,
#     xaxis = dict(nticks=4, range=[.3,.6]),
#     yaxis = dict(nticks=4, range=[.0,.9]),
    )
stroke_index = 0
lst = []
for path_index in range(20):
    d = dModel(z_path_list[path_index])
    xx_hat =gModel(d)
    x_stroke = ga.unFlatten(xx_hat)
    gs = np.array([x_stroke[stroke_index,:,0], x_stroke[stroke_index,:,1]]) #x,y stroke1
    lst.append(gs)

x_input_graph0 = ga.graphCurves(lst, layout=layout2D, dashRange=range(40,80), mode='markers+lines')
print(OutStroke.shape)
x_input_graph0

# x_input_graph0.write_image("./image/2D char vortex.png")

(104, 20)


In [24]:
la = np.array(lst)
pp_attractor = py.subplots.make_subplots(rows=1, cols=2,subplot_titles = [])

for i in [0,1,2,3,4,5,6,15,16,17,18,19]:
    x= la[i,0,:-1]
    y= la[i,1,:-1]
    u= la[i,0,1:]-x
    v= la[i,1,1:]-y

    fq = ff.create_quiver(x, y, u, v,
            scale=.8,
            arrow_scale=.3,
            name=i,
          )
    add_trace(pp_attractor,fq,row=1,col=1)

pp_attractor.layout = layout2D
pp_attractor
# pp_attractor.write_image("./image/2D char field.png")